<img src="http://hilpisch.com/tpq_logo.png" width="36%" align="right" style="vertical-align: top;">

# Natural Language Processing

**Basic Techniques and Algorithms**

_Illustrated based on the texts from three Apple press releases._

Dr Yves J Hilpisch | Michael Schwed

The Python Quants GmbH

## Data Retrieval

In [ ]:
import requests  

In [ ]:
sources = [
    'https://nr.apple.com/dE0b1T5G3u',  # iPad Pro
    'https://nr.apple.com/dE4c7T6g1K',  # MacBook Air
    'https://nr.apple.com/dE4q4r8A2A',  # Mac Mini
]  

In [ ]:
html = [requests.get(url).text for url in sources]  

## Preprocessing

In [ ]:
import sys
sys.path.append('../../modules/')
import nlp_functions as nlp  

In [ ]:
data = [nlp.clean_up_text(t) for t in html]  

In [ ]:
data[0][:500]  

## Basic Text Analysis

In [ ]:
import nltk

In [ ]:
nltk.__version__

In [ ]:
# to be executed once
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('vader_lexicon')

### Basic Tokenization and Vocabulary

In [ ]:
t = 'A sentence about NLP; and about ML.'

In [ ]:
nltk.word_tokenize(t)

In [ ]:
nltk.word_tokenize(data[0])[:10]  

In [ ]:
tokens = nltk.word_tokenize(' '.join(data))  

In [ ]:
text = nltk.Text(tokens)  

In [ ]:
text.count('ipad')  

In [ ]:
text.count('mojave')  

In [ ]:
text.collocations()  

In [ ]:
text.similar('performance')  

In [ ]:
text.common_contexts(['ipad', 'macbook'])  

In [ ]:
text.vocab()  

In [ ]:
text.concordance('macbook', width=70, lines=7)

In [ ]:
text.concordance('mojave', width=70, lines=7)

In [ ]:
words = sorted(set([w.lower() for w in tokens]))  

In [ ]:
len(words)  

In [ ]:
words[:7]  

### Improved Tokenization and Vocabulary

In [ ]:
t = 'A sentence about NLP; and about ML.'

In [ ]:
tokens = nlp.tokenize(t, min_char=3)
tokens

In [ ]:
%time tokens = nlp.tokenize(' '.join(data))  

In [ ]:
tokens[:7]  

In [ ]:
words = sorted(set([w.lower() for w in tokens]))  

In [ ]:
len(words)  

In [ ]:
words[:6]  

## Similarity

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
ts = [
    'A sentence about NLP; and about ML.',
    'Another one about NLP and algorithms.',
    'A text fragment about Apple.'
]  

In [ ]:
vec = TfidfVectorizer(stop_words='english')  

In [ ]:
mat = vec.fit_transform(ts)  

In [ ]:
mat.A.round(3)  

In [ ]:
df = pd.DataFrame(mat.A, columns=vec.get_feature_names())  

In [ ]:
df.round(3)  

In [ ]:
(mat * mat.T).A  

In [ ]:
tl = [' '.join(nlp.tokenize(t)) for t in data]  

In [ ]:
mat = vec.fit_transform(tl)  
mat  

In [ ]:
sm = (mat * mat.T).A  
sm  

In [ ]:
prs = ['ipad', 'air', 'mini']  

In [ ]:
df = pd.DataFrame(sm, index=prs, columns=prs)  

In [ ]:
df  

## Word Clouds

In [ ]:
for name, text in zip(prs, data):  
    nlp.generate_word_cloud(text, 35)#,
           # name='../../images/wc_{}.png'.format(name))  

## Topic Modeling

### NMF Clustering

In [ ]:
from sklearn.decomposition import NMF

In [ ]:
vec = TfidfVectorizer(stop_words='english')  

In [ ]:
mat = vec.fit_transform(data)  

In [ ]:
nmf = NMF(n_components=len(data))  

In [ ]:
nmf.fit(mat)  

In [ ]:
nmf.components_.round(3)

In [ ]:
feature_names = vec.get_feature_names()

In [ ]:
n_words = 8  

In [ ]:
for i, topic in enumerate(nmf.components_[:n_words]):
    print('TOPIC %d'% (i))
    print(60 * '=')
    print(', '.join([feature_names[i] for i in
                     topic.argsort()[:-n_words-1:-1]]) + '\n')  

### KMeans Clustering

In [ ]:
import collections
from sklearn.cluster import KMeans

In [ ]:
n_clusters = len(data)

In [ ]:
km = KMeans(n_clusters=n_clusters)  

In [ ]:
km.fit(mat)  

In [ ]:
n_kw = 7

In [ ]:
sorted_centroids = km.cluster_centers_.argsort()[:, ::-1]
words = vec.get_feature_names()
kw_list = []
for c in range(n_clusters):
    keywords = []
    for w in sorted_centroids[c, :n_kw]:
        keywords.append(words[w])
    kw_list.append(keywords)

In [ ]:
kw_list

In [ ]:
kw_df = pd.DataFrame(index=['topic_{}'.format(i) for i in range(n_clusters)], 
                           columns=['keyword_{}'.format(i) for i in range(n_kw)],
                           data=kw_list)

In [ ]:
kw_df

## Sentiment Analysis

In [ ]:
import warnings; warnings.simplefilter('ignore')

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
sid = SentimentIntensityAnalyzer()  

In [ ]:
p = 'The product is amazing. I love it.'  

In [ ]:
sid.polarity_scores(p)  

In [ ]:
n = 'The product is of low quality. I cannot recommend it.'  

In [ ]:
sid.polarity_scores(n)  

In [ ]:
sid.polarity_scores(' '.join(data))  

In [ ]:
scores = [sid.polarity_scores(l) for l in data]  

In [ ]:
scores 

## Summarization

In [ ]:
import pprint
from gensim.summarization import keywords
from gensim.summarization.summarizer import summarize

In [ ]:
for i, tokens in enumerate(tl):
    print('\nTEXT {}\n'.format(i) + 50 * '=')
    kws = keywords(tokens, words=8, split=False,
                   lemmatize=True, scores=True)  
    pprint.pprint(kws)

In [ ]:
texts = [nlp.clean_up_html(h) for h in html]  

In [ ]:
for i, text in enumerate(texts):
    print('\nTEXT {}\n'.format(i) + 50 * '=')
    print(summarize(text, word_count=45))  

<img src="http://hilpisch.com/tpq_logo.png" width="36%" align="right" style="vertical-align: top;">